In [1]:
import scanpy as sc
data_path = '../dataset/Chromium_FFPE_Human_Breast_Cancer_Chromium_FFPE_Human_Breast_Cancer_count_sample_filtered_feature_bc_matrix.h5'
sc_data = sc.read_10x_h5(data_path)
sc_data.X = sc_data.X.toarray()
st_data_path = '../dataset/Xenium_breast_cancer_sample1_replicate1.h5ad'
st_data = sc.read_h5ad(st_data_path)
st_data
# 检查 sc_data.var_names 是否有重复项
duplicates = sc_data.var_names[sc_data.var_names.duplicated()]
print(f"重复的基因: {duplicates}")
# 删除 sc_data.var_names 中的重复基因
sc_data = sc_data[:, ~sc_data.var_names.duplicated()]
st_data.X = st_data.X.toarray()
import numpy as np
shared_genes = np.intersect1d(st_data.var_names, sc_data.var_names)
st_data = st_data[:, shared_genes]
sc.pp.filter_cells(st_data, min_counts=1)
sc.pp.filter_cells(sc_data, min_counts=1)
sc_data = sc_data.copy()
st_data = st_data.copy()

st_data.obs['domain_id'] = 0
st_data.obs['domain_id'] = st_data.obs['domain_id'].astype('category')
st_data.obs['source'] = 'st'

sc_data.obs['source'] = 'sc'
sc_data.obs['domain_id'] = 1
sc_data.obs['domain_id'] = sc_data.obs['domain_id'].astype('category')

# Define genes to hold out for validation
holdout_genes = ['POSTN', 'IL7R', 'ITGAX', 'ACTA2', 'KRT15', 'VWF', 'FASN', 'CEACAM6']

# Store the original expression values of holdout genes for later comparison
holdout_gene_data = {}
for gene in holdout_genes:
    if gene in st_data.var_names:
        gene_idx = st_data.var_names.get_loc(gene)
        holdout_gene_data[gene] = st_data.X[:, gene_idx].copy()
        print(f"Stored original expression for {gene}")
    else:
        print(f"Gene {gene} not found in spatial data")

# Create a copy of the spatial data without the holdout genes
st_data_holdout = st_data.copy()
st_data_holdout = st_data_holdout[:, ~st_data_holdout.var_names.isin(holdout_genes)].copy()
print(f"Original gene count: {st_data.n_vars}, After holdout: {st_data_holdout.n_vars}")




adata_cm = st_data_holdout.concatenate(sc_data, join='inner', batch_key='domain_id')

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scvi.external import GIMVI
GIMVI.setup_anndata(st_data_holdout, batch_key='source')
GIMVI.setup_anndata(sc_data)
import uniport as up
sc.pp.normalize_total(adata_cm)
sc.pp.log1p(adata_cm)
up.batch_scale(adata_cm)
print(adata_cm)
sc.pp.normalize_total(st_data)
sc.pp.log1p(st_data)
up.batch_scale(st_data)
print(st_data)
sc.pp.normalize_total(sc_data)
sc.pp.log1p(sc_data)
up.batch_scale(sc_data)
print(sc_data)
adatas = [st_data, sc_data]
adata = up.Run(adatas=adatas, adata_cm=adata_cm, model_info=True)
adata_predicted = up.Run(adata_cm=st_data_holdout, out='predict', pred_id=1)
adata_predicted.obsm['predict']

adata_predicted.obsm['predict'].shape
import anndata as ad
adata_imputed = ad.AnnData(
    X=adata_predicted.obsm['predict'],
    obs=st_data.obs,
    uns=st_data.uns,
)
adata_imputed.var_names = sc_data.var_names
print(adata_imputed.var_names)

# Calculate Pearson correlation for holdout genes
correlations = {}
for gene in holdout_genes:
    if gene in adata_imputed.var_names and gene in holdout_gene_data:
        imputed_values = adata_imputed[:, gene].X.flatten()
        original_values = holdout_gene_data[gene]
        corr = np.corrcoef(imputed_values, original_values)[0, 1]
        correlations[gene] = corr
        print(f"Pearson correlation for {gene}: {corr:.4f}")

# Print average correlation
if correlations:
    avg_corr = np.mean(list(correlations.values()))
    print(f"Average correlation across holdout genes: {avg_corr:.4f}")



adata_imputed.X = adata_imputed.X.astype(np.float16)
adata_imputed.write_h5ad('../dataset/uniport_imputed_Xenium_breast_cancer_sample1_replicate1_hold_out.h5ad')

/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


重复的基因: Index(['TBCE', 'HSPA14', 'TMSB15B'], dtype='object')


/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number


/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number


...

Device: cuda


/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/uniport/function.py:414: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  batches = adata_cm.obs[source_name][np.sort(idx)]


Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1', 'HES4', 'ISG15',
       'AGRN', 'RNF223', 'C1orf159',
       ...
       'MT-ND2', 'MT-CO2', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4',
       'MT-ND5', 'MT-ND6', 'MT-CYB'],
      dtype='object', length=18079)
Pearson correlation for POSTN: 0.6544
Pearson correlation for IL7R: 0.5755
Pearson correlation for ITGAX: 0.5581
Pearson correlation for ACTA2: 0.5162
Pearson correlation for KRT15: 0.4309
Pearson correlation for VWF: 0.6681
Pearson correlation for FASN: 0.5770


Pearson correlation for CEACAM6: 0.4679
Average correlation across holdout genes: 0.5560


... storing 'source' as categorical
